In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#Emptying the Cache memory present in the CUDA
import torch
torch.cuda.empty_cache()

## Importing the necessary packages and initialising various parameters ##
import torch
import os
from torch.utils.data import Dataset
import cv2
from torch.nn import ConvTranspose2d
from torch.nn import Conv2d
from torch.nn import MaxPool2d
from torch.nn import Module
from torch.nn import ModuleList
from torch.nn import ReLU
from torchvision.transforms import CenterCrop
from torch.nn import functional as F
from torch.nn import BCEWithLogitsLoss
from torch.optim import Adam
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from torchvision import transforms
from imutils import paths
from tqdm import tqdm
import matplotlib.pyplot as plt
import time
import matplotlib.pyplot as plt
import numpy as np
import albumentations as A
from albumentations.pytorch import ToTensorV2
from skimage.io import imread, imsave
from skimage.transform import resize
import torchvision
import torchvision.transforms.functional as G
from google.colab.patches import cv2_imshow
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
from torch.utils import model_zoo
from torchvision import models
from torch.optim.lr_scheduler import StepLR

# Initializing learning rate, number of epochs to train and the batch size
learning_rate= 0.001
Number_of_epochs = 30
Batch_size = 2

# Define threshold to filter weak predictions
Threshold= 0.5

#Define the test split
Test_split= 0.15

#Determine the device to be used for training and evaluation
Device= "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
## Dataset loading ##

# Transformations of the image to desired size and convert to tensor
transforms = transforms.Compose([transforms.ToPILImage(),transforms.Resize((256,256)),transforms.ToTensor()])


class SegmentationDataset(Dataset):
	def __init__(self, imagePaths, maskPaths, transforms):
		# store the image and mask filepaths, and augmentation transforms
		self.imagePaths = imagePaths
		self.maskPaths = maskPaths
		self.transforms = transforms

	def __len__(self):
		# return the number of total samples contained in the dataset
		return len(self.imagePaths)

	def __getitem__(self, idx):
		imagePath = self.imagePaths[idx]  # grab the image path from the current index
		maskPath=self.maskPaths[idx]
		image = cv2.imread(imagePath,0)
		mask = cv2.imread(maskPath,0)
	 # check to see if we are applying any transformations
		if self.transforms is not None:
			image = self.transforms(image) # apply the transformations to both image and its mask
			mask = self.transforms(mask)
		# return a tuple of the image and its mask
		return (image, mask)


# load the image and mask filepaths in a sorted manner
imagePaths = sorted(list(paths.list_images('/content/drive/MyDrive/IIIT/Projects/Brain_Hemorrage_Dataset/Segmentation/Data/Image')))
maskPaths = sorted(list(paths.list_images('/content/drive/MyDrive/IIIT/Projects/Brain_Hemorrage_Dataset/Segmentation/Data/Mask')))
# partition the data into training and testing splits using 85% of the data for training and the remaining 15% for testing
split = train_test_split(imagePaths, maskPaths,test_size=0.15, random_state=42)
# unpack the data split
(trainImages, testImages) = split[:2]
(trainMasks, testMasks) = split[2:]


# create the train and test datasets
trainDS = SegmentationDataset(imagePaths=trainImages, maskPaths=trainMasks,transforms=transforms)
testDS = SegmentationDataset(imagePaths=testImages, maskPaths=testMasks,transforms=transforms)

print(f"[INFO] found {len(trainDS)} examples in the training set...")
print(f"[INFO] found {len(testDS)} examples in the test set...")

# create the training and test data loaders
trainLoader = DataLoader(trainDS, shuffle=True,batch_size=Batch_size,num_workers=os.cpu_count())
testLoader = DataLoader(testDS, shuffle=False,batch_size=Batch_size,num_workers=os.cpu_count())

[INFO] found 249 examples in the training set...
[INFO] found 44 examples in the test set...


In [4]:
import torch
import torch.nn as nn

class CustomSegmentationModel(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(CustomSegmentationModel, self).__init__()
        self.encoder_conv1 = nn.Conv2d(in_channels, 32, kernel_size=3, padding=1)
        self.encoder_relu1 = nn.ReLU(inplace=True)
        self.encoder_conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.encoder_relu2 = nn.ReLU(inplace=True)
        self.encoder_pool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.middle_conv1 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.middle_relu1 = nn.ReLU(inplace=True)
        self.middle_conv2 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.middle_relu2 = nn.ReLU(inplace=True)

        self.decoder_conv1 = nn.Conv2d(128, 64, kernel_size=3, padding=1)
        self.decoder_relu1 = nn.ReLU(inplace=True)
        self.decoder_conv2 = nn.Conv2d(64, out_channels, kernel_size=3, padding=1)
        self.decoder_relu2 = nn.ReLU(inplace=True)
        self.decoder_upsample = nn.Upsample(scale_factor=2,mode='bilinear', align_corners=True)

    def forward(self, x):
        x1 = self.encoder_conv1(x)
        x1 = self.encoder_relu1(x1)
        x2 = self.encoder_conv2(x1)
        x2 = self.encoder_relu2(x2)
        x3 = self.encoder_pool(x2)

        x4 = self.middle_conv1(x3)
        x4 = self.middle_relu1(x4)
        x5 = self.middle_conv2(x4)
        x5 = self.middle_relu2(x5)

        x6 = self.decoder_conv1(x5)
        x6 = self.decoder_relu1(x6)
        x7 = self.decoder_conv2(x6)
        output = self.decoder_relu2(x7)
        output = self.decoder_upsample(output)

        return output

# Hyperparameters
num_classes = 1  # Background and Foreground
in_channels = 1  # Grayscale images
out_channels = num_classes

# Initialize the model
model = CustomSegmentationModel(in_channels, out_channels)

class DiceLoss(Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceLoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):

        #comment out if your model contains a sigmoid or equivalent activation layer
        #inputs = torch.sigmoid(inputs)

        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)

        intersection = (inputs * targets).sum()
        dice_loss = 1-(2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)

        return dice_loss

class IoULoss(Module):
    def __init__(self, weight=None, size_average=True):
        super(IoULoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):

        #comment out if your model contains a sigmoid or equivalent activation layer
        #inputs = torch.sigmoid(inputs)

        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)

        #intersection is equivalent to True Positive count
        #union is the mutually inclusive area of all labels & predictions
        intersection = (inputs * targets).sum()
        total = (inputs + targets).sum()
        union = total - intersection

        IoU = (intersection + smooth)/(union + smooth)

        return 1 - IoU

In [8]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import numpy as np

# Define your CustomSegmentationModel, DiceLoss, and IoULoss here

# Hyperparameters
num_classes = 1
in_channels = 1
out_channels = num_classes
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize the model
model = CustomSegmentationModel(in_channels, out_channels).to(device)

# Loss functions
dice_loss_fn = DiceLoss()
iou_loss_fn = IoULoss()

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(Number_of_epochs):
    model.train()

    # Initialize metrics
    total_dice_loss = 0.0
    total_iou_loss = 0.0
    total_accuracy = 0.0
    total_precision = 0.0
    total_recall = 0.0

    for batch_idx, (images, targets) in enumerate(trainLoader):  # Replace with your train DataLoader
        images, targets = images.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = model(images)

        dice_loss = dice_loss_fn(outputs, targets)
        iou_loss = iou_loss_fn(outputs, targets)

        total_loss = dice_loss + iou_loss

        total_loss.backward()
        optimizer.step()

        # Calculate additional metrics
        predicted_masks = (outputs.sigmoid().squeeze() >= 0.5).float()
        accuracy = (predicted_masks == targets).float().mean()

        tp = ((predicted_masks == 1) & (targets == 1)).sum().item()
        fp = ((predicted_masks == 1) & (targets == 0)).sum().item()
        fn = ((predicted_masks == 0) & (targets == 1)).sum().item()

        precision = tp / (tp + fp + 1e-5)
        dice_coefficient = (2 * tp) / (2 * tp + fp + fn + 1e-5)
        jaccard_index = tp / (tp + fp + fn + 1e-5)

        total_dice_loss += dice_loss.item()
        total_iou_loss += iou_loss.item()
        total_accuracy += accuracy.item()
        total_precision += precision

    # Calculate average metrics for the epoch
    avg_dice_loss = total_dice_loss / len(trainLoader)
    avg_iou_loss = total_iou_loss / len(trainLoader)
    avg_accuracy = total_accuracy / len(trainLoader)
    avg_precision = total_precision / len(trainLoader)

    print(f"Epoch [{epoch+1}/{Number_of_epochs}] "
          f"Avg Dice Loss: {avg_dice_loss:.4f} "
          f"Avg IoU Loss: {avg_iou_loss:.4f} "
          f"Avg Accuracy: {avg_accuracy:.4f} "
          f"Avg Precision: {avg_precision:.4f} ")

    model.eval()
    with torch.no_grad():
        for val_batch_idx, (val_images, val_targets) in enumerate(testLoader):  # Replace with your validation DataLoader
            val_images, val_targets = val_images.to(device), val_targets.to(device)

            val_outputs = model(val_images)

            val_dice_loss = dice_loss_fn(val_outputs, val_targets)
            val_iou_loss = iou_loss_fn(val_outputs, val_targets)

            # Calculate additional validation metrics

        print(f"Validation Epoch [{epoch+1}/{Number_of_epochs}] --- Loss: {val_dice_loss + val_iou_loss:.4f} "
              f"--- Validation Dice Loss: {val_dice_loss:.4f} --- Validation IoU Loss: {val_iou_loss:.4f} "
              f"--- Validation Metrics: ...")  # Print validation metrics

Epoch [1/30] Avg Dice Loss: 0.9977 Avg IoU Loss: 0.9977 Avg Accuracy: 0.0038 Avg Precision: 0.0039 
Validation Epoch [1/30] --- Loss: 1.9934 --- Validation Dice Loss: 0.9967 --- Validation IoU Loss: 0.9967 --- Validation Metrics: ...
Epoch [2/30] Avg Dice Loss: 0.9980 Avg IoU Loss: 0.9980 Avg Accuracy: 0.0039 Avg Precision: 0.0039 
Validation Epoch [2/30] --- Loss: 1.9934 --- Validation Dice Loss: 0.9967 --- Validation IoU Loss: 0.9967 --- Validation Metrics: ...
Epoch [3/30] Avg Dice Loss: 0.9980 Avg IoU Loss: 0.9980 Avg Accuracy: 0.0038 Avg Precision: 0.0039 
Validation Epoch [3/30] --- Loss: 1.9934 --- Validation Dice Loss: 0.9967 --- Validation IoU Loss: 0.9967 --- Validation Metrics: ...
Epoch [4/30] Avg Dice Loss: 0.9978 Avg IoU Loss: 0.9978 Avg Accuracy: 0.0039 Avg Precision: 0.0040 
Validation Epoch [4/30] --- Loss: 1.9934 --- Validation Dice Loss: 0.9967 --- Validation IoU Loss: 0.9967 --- Validation Metrics: ...
Epoch [5/30] Avg Dice Loss: 0.9980 Avg IoU Loss: 0.9980 Avg Accu